In [ ]:
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup as bs
import sqlite3

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# setup sqlite database
connectDB = sqlite3.connect('news.sqlite')
cur = connectDB.cursor()


In [ ]:
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup as bs
import sqlite3

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# setup sqlite database
connectDB = sqlite3.connect('jpnn2016.sqlite')
intialize = connectDB.cursor()
cur = connectDB.cursor()

try:
    intialize.execute('''CREATE TABLE "Source" (
	"id"	INTEGER,
	"title"	TEXT,
	"firstparagraph"	TEXT,
	"date"	TEXT,
	"location"	TEXT,
	"latitude"	TEXT,
	"longitude"	TEXT,
	"full_address"	TEXT,
	"link"	TEXT,
	"kecamatan"	TEXT,
	"kota_kabupaten"	TEXT,
	"provinsi"	TEXT,
	PRIMARY KEY("id" AUTOINCREMENT)
    )''')
    print("success to create table")

except:
    print("fail to create table")



In [ ]:
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup as bs
import sqlite3

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# setup sqlite database
connectDB = sqlite3.connect('jpnn2016.sqlite')
intialize = connectDB.cursor()
cur = connectDB.cursor()

# scraping parameters
startyear = 2016
endyear = 2016
startmonth = 12
endmonth = 12
startday = 29
endday = 31

# looping through years, months, days
print("begin...")
for year in range(startyear,endyear+1):
  for month in range(startmonth,endmonth+1):
    for day in range(startday,endday+1):
      page = 1
      redirected = False

      # looping through every pages in index
      while True:

        # skip to next loop if page is redirected
        if redirected == True:
          break
        
        # trying until the request replied
        while True:
          try:
            concatedURL = 'https://www.jpnn.com/indeks?id=276&d='+str(day)+'&m='+str(month)+'&y='+str(year)+'&tab=all&page='+str(page)
            reqIndex = urllib.request.Request(concatedURL, headers={'User-Agent': 'Mozilla/5.0'})
            indeks = urllib.request.urlopen(reqIndex, context=ctx)
          except:
            print("retrying connection...")
            time.sleep(1)
            continue
          break
        
        # breaking the loop if redirected and pass redirected parameter to break parent loop
        if concatedURL != indeks.geturl(): 
          print(concatedURL+" is redirected to "+indeks.geturl())
          redirected = True
          break
        
        # selecting links in indeks
        indeks = indeks.read()
        parsedIndex = bs(indeks, 'html.parser')
        linkSelector = '#content-utama > div.kolom-kiri > div.border-box > div > ul > li > div.content-description > div > h2 > a'
        links = parsedIndex.select(linkSelector)

        # iterate every links
        for link in links:
          title = link.get_text()
          contentURL = urllib.parse.quote(link['href'], safe="http://")

          # same tricks as earlier request
          while True:
            try:
              reqContent = urllib.request.Request(contentURL, headers={'User-Agent': 'Mozilla/5.0'}) 
              print("opening "+title)
            except:
              print("retrying connection...")
              time.sleep(1)
              continue
            break
          
          # obtaining location data
          try:
            content = urllib.request.urlopen(reqContent, context=ctx).read()
            parsedContent = bs(content, 'html.parser')
            locationSelector = '#content-utama > div.kolom-kiri > div.border-box > div > div.content-scroll > div.page-content > div > p > a'
            locationData = parsedContent.select(locationSelector)
          except:
            print("cannot parse")
            break

          # if not available, assign "None" value
          try: 
            location = locationData[0].contents[0]
          except: 
            location = "None"
            pass
          
          # obtaining first page content
          fpSelector = '#content-utama > div.kolom-kiri > div.border-box > div > div.content-scroll > div.page-content > div > p'
          fpData = parsedContent.select(fpSelector)

          # if no content, use None
          try:
            fp = fpData[0].get_text()
          except:
            fp = "None"

          # insert new row into database
          try:
            date = str(year)+'-'+str(month)+'-'+str(day)
            link = link['href']
            cur.execute('INSERT INTO Source (title, firstparagraph, date, location, link) VALUES (?, ?, ?, ?, ?)', (title, fp, date, location, link))
            connectDB.commit()
          except:
            print("error updating database")

        print(str(day)+'/'+str(month)+'/'+str(year)+" page "+str(page)+' was parsed')

        # find whether there is more pages
        nextButton = parsedIndex.find_all(rel='next') 
        if nextButton == []:
          break
        else:
          page += 1
                

In [ ]:
# DATA PROCESSING
import sqlite3


# setup sqlite database
connectDB = sqlite3.connect('jpnn2016.sqlite')

# edit date
from datetime import date as cd

cur2 = connectDB.cursor()
cur3 = connectDB.cursor()

cur2.execute('SELECT date,link FROM Source')
count = 0

print("begin...")

for row in cur2:
    link = row[1]
    time = row[0].split("-")
    year = int(time[0])
    month = int(time[1])
    day = int(time[2])
    try:
        date = cd(year, month, day).isoformat()
        cur3.execute('UPDATE Source SET date = ? WHERE link = ?', (date, link))
        connectDB.commit()
        print(link+" updated")
    except ValueError:
        # deleting date that out of range caused by parsing error 
        # (e.g. Feb 30th news is impossible)
        cur3.execute('DELETE FROM Source WHERE link = ?', (link,))
        connectDB.commit()
        print(link + "deleted")


    

In [ ]:
# substract jpnn.com and google analytics tag from content
import sqlite3

connectDB = sqlite3.connect('jpnn2016.sqlite')
findjpnn = connectDB.cursor()
findjpnn.execute("SELECT firstparagraph,link FROM Source")

recontent = connectDB.cursor()

checkword = ('jpnn.com - ','JPNN.Com - ','JPNN.com - ','jpnn.com - ','JPNN.com - ','JPNN.com--','jpnn.com, ','JPNN.com – ','JPNN.com- ','JPNN.com – ','JPNN.Com – ','JPNN.Com – ','JPNN.Com – ','JPNN.com — ','jpnn.com - ','jpnn.com- ','jpnn.com -','JPNN.com ','JPNN.com - ','JPNN.com — ','JPNN.Com - ')
gtag = "googletag.cmd.push(function() { googletag.display('div-gpt-ad-1556314520901-0'); });"

count = 0

print("begin loop...")
for row in findjpnn:
    content = row[0]
    link = row[1]

    if (content == "") or (content == None): 
        recontent.execute("DELETE FROM Source WHERE link = ?",(link,))
        connectDB.commit()
        print(link+" is deleted")
    else:    
        for word in checkword:
            if (content.find(word) != -1) or (content.find(gtag) != -1):
                content = content.replace(word,"")
                content = content.replace(gtag,"")
                recontent.execute("UPDATE Source SET firstparagraph = ? WHERE link = ?",(content, link))
                connectDB.commit()
                update = True
            else:
                update = False
        if update == True: print(link+" is updated")
    
    # count += 1
    # if count == 401: break
print("done")


In [ ]:
# Attempt to get location data from content paragraph using regular expression
cur4 = connectDB.cursor()
cur5 = connectDB.cursor()

import re

cur4.execute('SELECT firstparagraph,link FROM Source')

count = 0


for row in cur4:
    locationdata = row[0]
    linkdata = row[1]
    locationdata = re.findall('^\s([A-Z]+.[A-Z]+)\s-',locationdata) or re.findall('^([A-Z]+.[A-Z]+)\s-',locationdata) or re.findall('^([A-Z]+.[A-Z]+)-',locationdata) or re.findall('^([A-Z]+.[A-Z]+)–',locationdata) or re.findall('^([A-Z]+.[A-Z]+)\s–',locationdata) or re.findall('^([A-Z]+.[A-Z]+)—',locationdata) or re.findall('^([A-Z]+.[A-Z]+)\s—',locationdata) or re.findall('^([A-Z]+.[A-Z]+)\s\W',locationdata)
    if locationdata == []: 
        cur5.execute('UPDATE Source SET location = ? WHERE link = ?', ('None',linkdata,))
        nolocation = True
        # print(linkdata)
        connectDB.commit()
    else:
        cur5.execute('UPDATE Source SET location = ? WHERE link = ?', (locationdata[0],linkdata,))
        connectDB.commit()
        print(locationdata[0]+" "+linkdata)
        
    # if nolocation == True: print(row[1]+" no location")

print("done")

In [ ]:
# delete if not uppercase (location not automatically determined)
# locupper = connectDB.cursor()
# locupper.execute("SELECT location,link FROM Source")
# deleterow = connectDB.cursor()

# import re

# count = 0
# for row in locupper:
#     location = row[0]
#     link = row[1]
#     location = re.findall('^([A-Z][A-Z]+)',location)
#     if location == []: 
#         deleterow.execute("DELETE FROM Source WHERE link = ?",(link,))
#         connectDB.commit()
#         count += 1
    

In [ ]:
# print(str(count)+" row deleted")


In [ ]:
# retrieve json location data using nominatim(osm) and insert it into province,city

# import urllib.request, urllib.parse
# import json

# address = 
# address =urllib.parse.quote(address)

# api_url = 'https://nominatim.openstreetmap.org/?addressdetails=1&q='+address+'&format=json&limit=1'


# openUrl = urllib.request.urlopen(api_url)
# decodedData = json.loads(openUrl.read())
# try: print("Kecamatan: "+decodedData[0]['address']['county']) 
# except: pass
# try: print("Kabupaten: "+decodedData[0]['address']['town'])
# except:pass
# try: print("Kota: "+decodedData[0]['address']['city'])
# except:pass
# try: print("Kabupaten: "+decodedData[0]['address']['state_district'])
# except:pass
# try: print("Provinsi: "+decodedData[0]['address']['state'])
# except:pass
# try: print("Lat: "+decodedData[0]['lat'])
# except:pass
# try: print("Lon: "+decodedData[0]['lon'])
# except:pass




In [8]:
# try using google geocode api
import apikey as key
import urllib.request, urllib.parse
import json
import sqlite3


connectDB = sqlite3.connect('jpnn2016.sqlite')
selectlocation = connectDB.cursor()
addLocation = connectDB.cursor()

selectlocation.execute("SELECT location,link FROM Source")

count = 0 

print("begin loop...")
for row in selectlocation:
    location = row[0]
    link = row[1]

    apikey = key.apikey
    apiUrl = 'https://maps.googleapis.com/maps/api/geocode/json?'

    params = dict()
    params['key'] = apikey
    params['address'] = location
    params['language'] = 'id'
    params['region'] = 'id'

    url = apiUrl + urllib.parse.urlencode(params)
    openUrl = urllib.request.urlopen(url)
    decodedData = json.loads(openUrl.read())
    
    try:
        data = decodedData['results'][0]['address_components']

        idkecamatan = ['administrative_area_level_3']
        idkota = ['administrative_area_level_2']
        idprovinsi = ['administrative_area_level_1']

        kecamatan = filter(lambda x: len(set(x['types']).intersection(idkecamatan)), data)
        kota = filter(lambda x: len(set(x['types']).intersection(idkota)), data)
        provinsi = filter(lambda x: len(set(x['types']).intersection(idprovinsi)), data)

        for data in kecamatan:
            kecamatan = data['long_name']
        for data in kota:
            kota = data['long_name']
        for data in provinsi:
            provinsi = data['long_name']

        def getKecamatan():
            if type(kecamatan) != str: datakecamatan = "None"
            else: datakecamatan = kecamatan
            return datakecamatan

        def getKota():
            if type(kota) != str: datakota = "None"
            else: datakota = kota
            return datakota

        def getProvinsi():
            if type(provinsi) != str: dataprovinsi = "None"
            else: dataprovinsi = provinsi
            return dataprovinsi

        def getFullAddress():
            full_address = decodedData['results'][0]['formatted_address']
            if type(full_address) != str: full_address = "None"
            return full_address

        def getLat():
            lat = decodedData['results'][0]['geometry']['location']['lat']
            if type(lat) != float: lat = "None"
            return lat

        def getLon():
            lon = decodedData['results'][0]['geometry']['location']['lng']
            if type(lon) != float: lon = "None"
            return lon

        try:
            query = "UPDATE Source SET kecamatan=?,kota_kabupaten=?,provinsi=?,latitude=?,longitude=?,full_address=? WHERE link=?"
            addLocation.execute(query,(getKecamatan(), getKota(), getProvinsi(), getLat(), getLon(), getFullAddress(), link))
            connectDB.commit()
            # print(link+" SUCCESS")
        except:
            print(link+" FAIL TO ADD TO DB")

    except:
        query = "UPDATE Source SET kecamatan=?,kota_kabupaten=?,provinsi=?,latitude=?,longitude=?,full_address=? WHERE link=?"
        addLocation.execute(query,("None","None","None","None","None","None", link))
        connectDB.commit()
        print(link+" EMPTY")

    count += 1
   
    # if count == 1: break

    if count % 500 == 0: print(link)





begin loop...
https://www.jpnn.com/news/teori-baru-lagi-mirna-diduga-korban-salah-sasaran
https://www.jpnn.com/news/muncikari-cabe-cabean-di-kedai-kopi-akhirnya-ditangkap-saat
https://www.jpnn.com/news/memalukan-polisi-peras-pengunjung-diskotek-uangnya-dipakai-mabuk
https://www.jpnn.com/news/pemuda-sakit-jiwa-hampir-habis-dihajar-massa
https://www.jpnn.com/news/ada-2-boneka-dalam-reka-ulang-kaburnya-anwar
https://www.jpnn.com/news/tiga-begal-dihajar-warga-lihat-tuhmukanya-remuk
https://www.jpnn.com/news/baku-tendang-di-atas-motor-malah-tewas EMPTY
https://www.jpnn.com/news/muncikari-para-imigran-ganteng-itu-akhirnya-diringkus
https://www.jpnn.com/news/polisi-palsu-kembali-beraksi-waspada-penipuan EMPTY
https://www.jpnn.com/news/miris-si-gadis-diperkosa-pacar-dan-teman-temannya
https://www.jpnn.com/news/ngaku-panti-pijat-tradisionaleh-digerebek-ternyata-lagi-indehoi
https://www.jpnn.com/news/maling-aneh-pintu-lemari-korban-dilem


In [ ]:
import sqlite3
import urllib.request, urllib.parse
import json


connectDB = sqlite3.connect('news.sqlite')
selectlocation = connectDB.cursor()
selectlocation.execute("SELECT location,link FROM Source")

addLocation = connectDB.cursor()



count = 0

for row in selectlocation:
    location = row[0]+" indonesia"
    link = row[1]
    location = urllib.parse.quote(location)

    api_url = 'https://nominatim.openstreetmap.org/?addressdetails=1&q='+location+'&format=json&limit=1'

    openUrl = urllib.request.urlopen(api_url)
    decodedData = json.loads(openUrl.read())


    def getKecamatan():
        try: kecamatan = decodedData[0]['address']['county']
        except: 
            kecamatan = "None"
            pass
        return kecamatan

    def getKota():
        kotakabupaten = "None"
        try: 
            kotakabupaten = decodedData[0]['address']['town']
        except:pass
        try: 
            kotakabupaten = decodedData[0]['address']['city']
        except:pass
        try: 
            kotakabupaten = decodedData[0]['address']['state_district']
        except:pass
        return kotakabupaten

    def getProvinsi():
        try: provinsi = decodedData[0]['address']['state']
        except: 
            provinsi = "None" 
            pass
        return provinsi

    def getLat():
        lat = "None"
        try: 
            lat = decodedData[0]['lat']
        except:pass
        return lat

    def getLon():
        lon = "None"
        try: 
            lon = decodedData[0]['lon']
        except:pass
        return lon

    kecamatan = getKecamatan()
    kota = getKota()
    provinsi = getProvinsi()
    lat = getLat()
    lon = getLon()

    count += 1
    if count == 2: break

